In [1]:
#Dongzhe Li 104434089 dongzheli@ucla.edu

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

import warnings
warnings.simplefilter('ignore', DeprecationWarning)


In [7]:
data = pd.read_csv('/Users/Dongzhe/Desktop/cs249/hw6/titanic_train.csv')
data.shape

(891, 12)

In [8]:
target = data.Survived.values
target.shape

(891,)

In [9]:
rich_features = pd.concat([data.get(['Fare', 'Pclass', 'Age','SibSp','Parch']),
                           pd.get_dummies(data.Sex, prefix='Sex'),
                           pd.get_dummies(data.Pclass, prefix='Pclass'),
                           pd.get_dummies(data.Embarked, prefix='Embarked')],
                          axis=1)

rich_features.count()

Fare          891
Pclass        891
Age           714
SibSp         891
Parch         891
Sex_female    891
Sex_male      891
Pclass_1      891
Pclass_2      891
Pclass_3      891
Embarked_C    891
Embarked_Q    891
Embarked_S    891
dtype: int64

In [10]:
rich_features.head(5)

,Fare,Pclass,Age,SibSp,Parch,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,7.2500,3,22,1,0,0,1,0,0,1,0,0,1
1,71.2833,1,38,1,0,1,0,1,0,0,1,0,0
2,7.9250,3,26,0,0,1,0,0,0,1,0,0,1
3,53.1000,1,35,1,0,1,0,1,0,0,0,0,1
4,8.0500,3,35,0,0,0,1,0,0,1,0,0,1


In [12]:
rich_features_no_male = rich_features.drop('Sex_male', 1)
rich_features_no_male.count()

Fare          891
Pclass        891
Age           714
SibSp         891
Parch         891
Sex_female    891
Pclass_1      891
Pclass_2      891
Pclass_3      891
Embarked_C    891
Embarked_Q    891
Embarked_S    891
dtype: int64

In [13]:
rich_features_no_male = rich_features_no_male.fillna(-1)
rich_features_no_male.head(6)

,Fare,Pclass,Age,SibSp,Parch,Sex_female,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,7.2500,3,22,1,0,0,0,0,1,0,0,1
1,71.2833,1,38,1,0,1,1,0,0,1,0,0
2,7.9250,3,26,0,0,1,0,0,1,0,0,1
3,53.1000,1,35,1,0,1,1,0,0,0,0,1
4,8.0500,3,35,0,0,0,0,0,1,0,0,1
5,8.4583,3,-1,0,0,0,0,0,1,0,1,0


In [14]:
from sklearn.cross_validation import train_test_split

X_train, X_test = train_test_split(rich_features_no_male.values)

In [15]:
from sklearn.preprocessing import Imputer

imputer = Imputer(strategy='mean', missing_values=-1)

imputer.fit(X_train)

Imputer(axis=0, copy=True, missing_values=-1, strategy='mean', verbose=0)

In [37]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import cross_val_score

imputer = Imputer(strategy='mean', missing_values=-1)

classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.05,
                                        subsample=.8, max_features=.25)

pipeline = Pipeline([
    ('imp', imputer),
    ('clf', classifier),
])
scores = cross_val_score(pipeline, rich_features_no_male, target, cv=10, n_jobs=4,
                         scoring='accuracy', )
print(scores.min(), scores.mean(), scores.max())

(0.7640449438202247, 0.8216189989785494, 0.898876404494382)


In [17]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(C=1)

In [18]:
from sklearn.cross_validation import cross_val_score

scores = cross_val_score(logreg, rich_features_no_male, target, cv=10, n_jobs=4,
                         scoring='accuracy')
print(scores.min(), scores.mean(), scores.max())

(0.7528089887640449, 0.79458943366246726, 0.8314606741573034)


In [19]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=150)
scores = cross_val_score(rf,rich_features_no_male, target, cv=5, n_jobs=4,
                         scoring='accuracy')
print(scores.min(), scores.mean(), scores.max())

(0.77094972067039103, 0.80366368098015928, 0.8370786516853933)


In [35]:
# choose the gradientboostingclassifier
from sklearn.grid_search import GridSearchCV

gb = GradientBoostingClassifier(n_estimators=100, subsample=.8)

params = {
    
    'learning_rate': [0.05, 0.1, 0.5,0.04],
    'max_features': [0.5, 1,0.25],
    'max_depth': [3, 4, 5],
}

gs = GridSearchCV(gb, params, cv=5, scoring='roc_auc', n_jobs=4)
learning_rate=0.01
gs.fit(rich_features_no_male, target)

gs.cv

5

In [36]:
gs.best_params_

{'learning_rate': 0.05, 'max_depth': 5, 'max_features': 0.25}

In [38]:
classifier.fit(rich_features_no_male,target)
#change the learning_rate to 0.05 above and fit the model with the data

GradientBoostingClassifier(init=None, learning_rate=0.05, loss='deviance',
              max_depth=3, max_features=0.25, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2, n_estimators=100,
              random_state=None, subsample=0.8, verbose=0,
              warm_start=False)

In [39]:
testdata = pd.read_csv('/Users/Dongzhe/Desktop/cs249/hw6/titanic_test.csv')

In [40]:
testdata.count()

PassengerId    418
Pclass         418
Name           418
Sex            418
Age            332
SibSp          418
Parch          418
Ticket         418
Fare           417
Cabin           91
Embarked       418
dtype: int64

In [41]:
rich_test_features = pd.concat([testdata.get(['Fare', 'Pclass', 'Age','SibSp','Parch']),
                           pd.get_dummies(testdata.Sex, prefix='Sex'),
                           pd.get_dummies(testdata.Pclass, prefix='Pclass'),
                           pd.get_dummies(testdata.Embarked, prefix='Embarked')],
                          axis=1)
rich_test_features = rich_test_features.drop('Sex_male', 1)
rich_test_features.count()

Fare          417
Pclass        418
Age           332
SibSp         418
Parch         418
Sex_female    418
Pclass_1      418
Pclass_2      418
Pclass_3      418
Embarked_C    418
Embarked_Q    418
Embarked_S    418
dtype: int64

In [43]:
mean_features = rich_test_features.dropna().mean()

imputed_features =rich_test_features.fillna(mean_features)
imputed_features.head(6)

,Fare,Pclass,Age,SibSp,Parch,Sex_female,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,7.8292,3,34.5,0,0,0,0,0,1,0,1,0
1,7.0000,3,47.0,1,0,1,0,0,1,0,0,1
2,9.6875,2,62.0,0,0,0,0,1,0,0,1,0
3,8.6625,3,27.0,0,0,0,0,0,1,0,0,1
4,12.2875,3,22.0,1,1,1,0,0,1,0,0,1
5,9.2250,3,14.0,0,0,0,0,0,1,0,0,1


In [44]:
import numpy as np
np.random.seed(12345)    
target_predicted = classifier.predict(imputed_features)
print(target_predicted )

[0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 1 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]
